In [1]:
!pip install tokenizers

     |████████████████████████████████| 6.8 MB 4.1 MB/s 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import torch
import torch.nn as nn

# Transformer로 챗봇을 만들어봅시다!
- Transformer에 문장을 입력하면, 그에 대한 대답을 하는 챗봇을 만들 것입니다.
- 중간중간 빈칸을 채워넣어가며 자신만의 챗봇 코드를 완성해보세요.

우선 챗봇 학습을 위한 데이터를 다운로드 받습니다.

In [3]:
# data download
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

In [ ]:
print('데이터 샘플의 개수 :', len(train_data))

데이터 샘플의 개수 : 11823


In [ ]:
train_data.head(-1)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11817,후회 없이 사랑하고 싶어,진심으로 다가가 보세요.,2
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2


## 한국어 데이터를 전처리하는 방법에 대해 알아봅시다.
- 우리는 네이버 영화리뷰 데이터에 tokenizer를 학습시켜 단어를 구분하도록 할 예정입니다. 
(챗봇 데이터에 비해 훨씬 많은 데이터를 포함하고 있습니다.)

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
naver_df = pd.read_table('ratings.txt')
naver_df = naver_df.dropna(how='any')
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [ ]:
naver_df

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
199995,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
199996,3302770,쓰.레.기,0
199997,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
199998,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [5]:
from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer(lowercase=False) # lowercase를 구분할지 여부를 선택합니다 (True: 대문자 무시(모두 소문자로 인식), False:대소문자 구분)


In [6]:
tokenizer # data를 준 것이 아닌 inicialize만 된 상태. 

Tokenizer(vocabulary_size=0, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=False, wordpieces_prefix=##)

In [7]:
data_file = 'naver_review.txt'
vocab_size = 30000 # 의미상으로 나눠지는 단위로 3만개를 설정. 3만개에 포함되지 못하면 Unknown으로 취급
limit_alphabet = 6000 # 알파벳 개수. 알파벳의 a-z/A-Z와 달리 한글은 가, 각, 간, ... 식으로 각각 취급
                      # 숫자도 0, 1, 2 .... 식으로 각각 나누어 취급 
min_frequency = 5

tokenizer.train(files=data_file,
                vocab_size=vocab_size,
                limit_alphabet=limit_alphabet,
                min_frequency=min_frequency,
                special_tokens=['[PAD]', '[START]', '[END]', '[UNK]'])

In [8]:
# vocab 저장
tokenizer.save_model('./') # ./는 현재 폴더
vocab_df = pd.read_fwf('vocab.txt', header=None)
vocab_df

,0
0,[PAD]
1,[START]
2,[END]
3,[UNK]
4,!
...,...
29995,맛도
29996,망하지
29997,망한다
29998,망가지


In [ ]:
vocab_df.head(30) # 정렬이 되었는지 숫자와 특수기호가 상위에 있는 것을 볼 수 있다.

In [9]:
# 챗봇 모델은 "정수 인코딩" 결과를 사용하여 모델을 작동시키고, 최종 예측 결과도 정수로 인코딩 된 결과를 내뱉습니다.
# 인코딩된 결과물을 decode 를 통해 해석 가능한 문장으로 바꿔줄 수 있습니다.
encoded = tokenizer.encode('챗봇이 잘 완성될까요?')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

토큰화 결과 : ['챗', '##봇', '##이', '잘', '완성', '##될까', '##요', '?']
정수 인코딩 : [2644, 4573, 3302, 2361, 13647, 20339, 3319, 34]
디코딩 : 챗봇이 잘 완성될까요?


# Transformer 모델을 만들어봅시다.

In [10]:
from torch import Tensor
import torch
import torch.nn as nn
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
## input embedding ## 
class PositionalEncoding(nn.Module): # 단어들을 구분하는 역할
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__() 
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)

        pos_embedding = torch.zeros((maxlen, emb_size)) # (문장의 최대길이, emb길이)
        pos_embedding[:, 0::2] = torch.sin(pos * den) # 짝수 위치에는 sin값
        pos_embedding[:, 1::2] = torch.cos(pos * den) # 홀수 위치에는 cos값
        pos_embedding = pos_embedding.unsqueeze(-2) # (max length, 1, embedding size) 
        # positional encoding + word embedding
        # word embedding
        # 1. token index 입력: [1, 5, 32, 64]
        # 2. token embedding (word embedding)
        # [1, 5, 32, 64] -> [-0.1 0.2 0.3]
        #                   [1.2 -0.1 0.2]
        #                   [            ]
        #                    ....
        # 4 x 3
        # (4, 3): 1개의 문장
        # (4, batch, 3): token embedding size
        # positional encoding
        # (4, 1, 3)
        # (4, batch, 3) + (4, 1, 3) = (4, batch, 3) = (sequence_length, batch_size, embedding_size)
        
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor): 
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
## positioning embedding ##
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
        # 1. token index 입력: [1, 5, 32, 64]
        # 2. token embedding (word embedding)
        # [1, 5, 32, 64] -> [-0.1 0.2 0.3]
        #                   [1.2 -0.1 0.2]
        #                   [            ]
        #                    ....
        # (4, 3): 1개의 문장
        # (length, batch, embedding size): token embedding size

    def forward(self, tokens: Tensor):
        # tokens = (batch, sequence length)
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        # ============== TODO ======================
        # torch.nn 에는 Transformer 모델이 이미 구현되어 있습니다. 
        # 이것을 이용해 챗봇 모델을 완성해봅시다.
        # ==========================================
        self.transformer = nn.Transformer(d_model=emb_size, 
                                nhead=nhead, # attaintion에서 쓰임 
                                num_encoder_layers=num_encoder_layers, # n-score (Nx)
                                num_decoder_layers=num_decoder_layers, 
                                dim_feedforward=dim_feedforward,
                                dropout=dropout)
        # Transformer의 output값을 vocabulary의 index를 나타내는 벡터로 바꾸어주는 선형 변환이 필요합니다.
        self.generator = nn.Linear(emb_size, tgt_vocab_size) 
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)
        # crossentropy를 하기때문에 따로 softmax 처리는 하지 않는다.

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        # ============== TODO ======================
        # 위에서 구현한 트랜스포머 모델에 입력된 데이터를 통과시켜봅시다.
        # ==========================================
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, 
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

- Transformer의 학습 중에는, 미래의 데이터를 볼 수 없도록 하는 마스크가 필요합니다.

In [11]:
PAD_IDX = 0
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [12]:
SRC_VOCAB_SIZE = vocab_size # 30000
TGT_VOCAB_SIZE = vocab_size # 30000
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

# 모델을 initialize 해줍니다.
model = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
model = model.to(DEVICE) # model 을 GPU로 보내줍니다.

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX) # ignore_index의 역할: padding 무시(pad로부터 오는 손실을 최소화하기 위함)
# PAD_IDX = 0으로 처음부분에 명시되어 있다. 

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [13]:
from torch.utils.data import Dataset
class ChatbotDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data # (numpy value)
        self.tokenizer = tokenizer # (BERTWordpieceTokenizer) 학습이 완료된 토크나이저

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y, _ = self.data[idx]
        # ============== TODO ==============
        # 1. 데이터를 토크나이즈 해보세요. (x,y 모두)
        # 2. start token 과 end token을 추가해보세요. (x,y 모두)
        # [SOS], [EOS] / [START], [END] -> 시작과 끝을 파악할 수 있도록 명시하는 토큰. input 때는 없어도 되지만 output 때는 필요하다.
        # start of sentence, end of sentence
        # 3. 토큰들을 torch tensor (long 타입)로 변환하고 return하세요.
        # ==================================

        x_tokens = self.tokenizer.encode(x).ids # index. list 형태
        y_tokens = self.tokenizer.encode(y).ids

        # hint: x_tokens 와 y_tokens 는 list type입니다.
        x_tokens.insert(0, 1) # start
        x_tokens.append(2) # end
        y_tokens.insert(0, 1)
        y_tokens.append(2) 

        return torch.LongTensor(x_tokens), torch.LongTensor(y_tokens)

In [14]:
dataset = ChatbotDataset(train_data.values, tokenizer)

In [15]:
x, y = dataset[100]

In [16]:
tokenizer.decode(x.tolist())

'거지됐어'

In [17]:
tokenizer.decode(y.tolist())

'밥 사줄 친구를 찾아 보세요 ~'

In [18]:
x, y

(tensor([   1, 7380, 3962, 3279,    2]),
 tensor([    1,  1586,  1812,  3422, 17757,  6478,  6492,    97,     2]))

In [19]:
# collate_fn 을 정의해줍니다.
# collate_fn 은 dataset으로부터 여러 item을 받아와 하나의 배치로 합칠 때, 어떻게 합칠지를 정의하는 부분입니다.

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
def collate_fn(batch):
    # batch: ((x1, y1), (x2, y2), ... )
    # x, y 

    # ============== TODO ==============
    # 우리의 목표는 X = [x1,x2,x3,...] , Y = [y1, y2, y3, ...] 형태의 tensor로 만드는 것입니다.
    # 이를 위해 2가지 해야할 일이 있습니다.
    # 1. batch 안에 있는 x 와 y를 각각의 list 에 모아주기.
    # 2. padding을 통해 동일한 길이로 만들어주고, 하나의 tensor로 통합하기.
    # 아래 빈칸을 채워 위 두가지를 진행해보세요.
    # ==================================
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(src_sample)
        tgt_batch.append(tgt_sample)
    # [x1, x2, x3, ...]: src_batch: list / x1, x2, x3,...: long type tensor
    # [y1, y2, y3, ...]: tgt_batch
    
    # padding_value는 pad 위치에 어떤 값을 넣을지를 정하는 값입니다.
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX) 
    return src_batch, tgt_batch

# dataloader 를 정의해줍니다.
batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, 
                         num_workers=2, collate_fn=collate_fn,
                          pin_memory=True, drop_last=True) 
# pin_memory: gpu에 보다 빠르게 올려주는 기능 
# drop_last: 마지막값을 버림. train할때는 돌리는 회차마다 버리는 값이 달라지나, test때는 항상 같으므로 사용해서는 안된다.


In [20]:
x,y = next(iter(data_loader))

Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


In [21]:
x.shape, y.shape

(torch.Size([14, 32]), torch.Size([15, 32]))

In [22]:
x[:,1]

tensor([   1, 7127, 5884, 3282, 7727, 5849,  611, 8093,    2,    0,    0,    0,
           0,    0])

In [23]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer, loss_fn, train_dataloader):
    model.train()
    losses = 0

    for src, tgt in train_dataloader:
        src = src.to(DEVICE) # default가 cpu이므로 gpu를 사용할 때 넣어주자
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1,:] # target은 마지막 하나를 빼는 이유: end 토큰은 예측할 필요가 없으므로...

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :] # 첫번째 단어는 빼고 로스를 계산하는 이유: start 토큰은 빼주어야하므로...
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        optimizer.step()

        # 출력용 
        losses += loss.item()

    return losses / len(train_dataloader)

In [24]:
EPOCHS = 10
for i in range(EPOCHS):
    epoch_loss = train_epoch(model, optimizer, loss_fn, data_loader)
    print('EPOCH {} LOSS {:.6f}'.format(i+1, epoch_loss))

EPOCH 1 LOSS 6.092745
EPOCH 2 LOSS 5.186776
EPOCH 3 LOSS 4.803029
EPOCH 4 LOSS 4.478619
EPOCH 5 LOSS 4.203460
EPOCH 6 LOSS 3.944638
EPOCH 7 LOSS 3.712794
EPOCH 8 LOSS 3.494999
EPOCH 9 LOSS 3.289377
EPOCH 10 LOSS 3.091527


In [25]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    # length 1, batch 1
    for i in range(max_len-1):
        # ys: length = 2, batch 1
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1) # (length, batch, output_dimension)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == 2:
            break
    return ys

def predict(model, tokenizer, src_sentence):
    model.eval() 
    src = tokenizer.encode(src_sentence).ids
    src.insert(0,1) # start
    src.append(2) # end
    src = torch.tensor(src).long().unsqueeze(1) # (sequence_length, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=1).flatten()
    return tokenizer.decode(tgt_tokens.cpu().tolist())

In [ ]:
# <start> 나는 집에 간다 . <end>

In [26]:
predict(model,tokenizer,'반가워')

'저도요.'

In [27]:
predict(model,tokenizer,'여행 가고 싶다.')

'좋은 곳으로 가보세요.'

# BERT 실습
- BERT는 Transformer의 encoder를 사용합니다.
- Transformer 구현에 대해 이미 알아보았으니, BERT를 직접 구현하지 않고 huggingface 라이브러리를 통해 간단하게 구현하는 방법에 대해 배우도록 하겠습니다.
- BERT를 이용해서 naver 영화리뷰 데이터의 점수를 분류하는 task를 진행해보겠습니다.
- https://huggingface.co/

In [28]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 42.1 MB/s 
     |████████████████████████████████| 61 kB 538 kB/s 
     |████████████████████████████████| 596 kB 51.1 MB/s 
     |████████████████████████████████| 3.3 MB 22.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.11.1
    Uninstalling tokenizers-0.11.1:
      Successfully uninstalled tokenizers-0.11.1


#### 영화리뷰 데이터에 대한 fine-tuning에 앞서, 간단하게 huggingface 라이브러리의 사용방법에 대해 익혀보도록 합시다.

- https://huggingface.co/transformers/v3.0.2/index.html

In [29]:
from transformers import BertTokenizer, BertForTokenClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # uncased: 대소문자 구분 안함
model = BertForTokenClassification.from_pretrained('bert-base-uncased') # from_pretrained: 

inputs = tokenizer.encode("Hello, my dog is cute", return_tensors="pt")
outputs = model(inputs)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [30]:
tokenizer.encode("Hello. my dog is cute", return_tensors = "pt") # pt: pytorch

tensor([[  101,  7592,  1012,  2026,  3899,  2003, 10140,   102]])

In [31]:
tokenizer.encode("Hello. my dog is cute", return_tensors = "pt").shape

torch.Size([1, 8])

In [32]:
tokenizer.decode(inputs.squeeze().tolist())
# CLS: 다음 문장이 이어진 문장인지 판단하는 토큰 
# SEP: 문장 두 개를 구분하는 토큰 

'[CLS] hello, my dog is cute [SEP]'

In [33]:
outputs 

# 이미 사전학습이 된 모델이라 바로 값을 확인할 수 있다.

TokenClassifierOutput([('logits', tensor([[[ 0.1386,  0.1260],
                                 [ 0.7114, -0.0359],
                                 [-0.1205, -0.1000],
                                 [-0.0928,  0.0951],
                                 [-0.2351, -0.2253],
                                 [-0.2358, -0.1069],
                                 [-0.3923, -0.1057],
                                 [-0.3681, -0.1728]]], grad_fn=<AddBackward0>))])

In [34]:
inputs.shape

torch.Size([1, 8])

- 사전 학습된 BERT의 mask 토큰에 대한 예측 결과를 확인해봅시다.

In [35]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
unmasker("The man worked as a [MASK].")

[{'score': 0.09747546911239624,
  'sequence': 'the man worked as a carpenter.',
  'token': 10533,
  'token_str': 'carpenter'},
 {'score': 0.052383411675691605,
  'sequence': 'the man worked as a waiter.',
  'token': 15610,
  'token_str': 'waiter'},
 {'score': 0.04962698742747307,
  'sequence': 'the man worked as a barber.',
  'token': 13362,
  'token_str': 'barber'},
 {'score': 0.037886083126068115,
  'sequence': 'the man worked as a mechanic.',
  'token': 15893,
  'token_str': 'mechanic'},
 {'score': 0.037680838257074356,
  'sequence': 'the man worked as a salesman.',
  'token': 18968,
  'token_str': 'salesman'}]

In [37]:
unmasker("The woman worked as a [MASK].")

[{'score': 0.21981535851955414,
  'sequence': 'the woman worked as a nurse.',
  'token': 6821,
  'token_str': 'nurse'},
 {'score': 0.1597413569688797,
  'sequence': 'the woman worked as a waitress.',
  'token': 13877,
  'token_str': 'waitress'},
 {'score': 0.11547300964593887,
  'sequence': 'the woman worked as a maid.',
  'token': 10850,
  'token_str': 'maid'},
 {'score': 0.03796879202127457,
  'sequence': 'the woman worked as a prostitute.',
  'token': 19215,
  'token_str': 'prostitute'},
 {'score': 0.030423851683735847,
  'sequence': 'the woman worked as a cook.',
  'token': 5660,
  'token_str': 'cook'}]

In [38]:
unmasker("I have a [MASK].")

[{'score': 0.0911906436085701,
  'sequence': 'i have a plan.',
  'token': 2933,
  'token_str': 'plan'},
 {'score': 0.04680357500910759,
  'sequence': 'i have a problem.',
  'token': 3291,
  'token_str': 'problem'},
 {'score': 0.030351677909493446,
  'sequence': 'i have a girlfriend.',
  'token': 6513,
  'token_str': 'girlfriend'},
 {'score': 0.029900116845965385,
  'sequence': 'i have a point.',
  'token': 2391,
  'token_str': 'point'},
 {'score': 0.027309242635965347,
  'sequence': 'i have a boyfriend.',
  'token': 6898,
  'token_str': 'boyfriend'}]

- BERT tokenizer와 model을 이용해 네이버 영화리뷰 데이터에 fine-tuning을 진행해봅시다.

In [39]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
naver_df = pd.read_table('ratings.txt')
naver_df = naver_df.dropna(how='any')
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [ ]:
naver_df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [ ]:
naver_df.tail()

,id,document,label
199995,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
199996,3302770,쓰.레.기,0
199997,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
199998,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0
199999,8548411,포풍저그가나가신다영차영차영차,0


In [ ]:
naver_df['label'].unique()

array([1, 0])

In [40]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased') # multilingual-BERT를 사용해보겠습니다.
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

inputs = tokenizer.encode("Hello, my dog is cute", return_tensors="pt")
outputs = model(inputs)

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [41]:
outputs

SequenceClassifierOutput([('logits',
                           tensor([[-0.4497, -0.2968]], grad_fn=<AddmmBackward0>))])

In [42]:
# label을 입력해주면 classification에 대한 loss도 자동으로 계산할 수 있습니다.
model(inputs, labels=torch.tensor([1]))

SequenceClassifierOutput([('loss', tensor(0.6196, grad_fn=<NllLossBackward0>)),
                          ('logits',
                           tensor([[-0.4497, -0.2968]], grad_fn=<AddmmBackward0>))])

In [43]:
tokenizer.decode(inputs.squeeze().tolist())

'[CLS] Hello, my dog is cute [SEP]'

In [44]:
train_data_idx = np.random.choice(range(len(naver_df)), size=len(naver_df)//5*4, replace=False)

In [ ]:
train_data_idx

array([171039,  78940, 157751, ...,  43223, 127356, 118903])

In [45]:
train_data = naver_df.iloc[train_data_idx][['document','label']].values
test_data = naver_df.iloc[~naver_df.index.isin(train_data_idx)][['document','label']].values

In [46]:
from torch.utils.data import Dataset
class ReviewDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx] # x: sentense / y: 0 or 1
        # 1. tokenizer.encode(x).ids
        # 2. torch long tensor
        
        # BERT tokenizer는 batch 단위로 한번에 token화 할 수 있습니다.
        # dataloder에서 얻어온 후 한번에 tokenize 해보도록 하겠습니다.
        # text는 list로, label은 long tensor로 리턴해줍니다.
        return x, torch.tensor(y).long()

In [47]:
train_dataset = ReviewDataset(train_data)
test_dataset = ReviewDataset(test_data)

In [48]:
batch_size = 32 # 크면 클수록 성능이 좋아진다. 대신 학습에 쓰이는 메모리가 문제...

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                         num_workers=2, collate_fn=None,
                          pin_memory=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, 
                         num_workers=2, collate_fn=None,
                          pin_memory=True, drop_last=False)

In [49]:
x, y = next(iter(train_loader))

In [50]:
x

['좋네요....저 밑에 우리나라 사형제도 폐지 됐다고 하는데폐지된거 아닙니다.... 다만, 형의 집행을 계속 시행하지 않고 있을뿐...',
 '병맛 쩌는데 볼때마다 끝까지 보게됨. 중독성있는 영화.ㅋㅋ',
 '예지원 연기가 좋았다.. 소재도 나름 신선했다. 의미전달에 약간의 아쉬움.',
 '이게.뭡니까! 록스코씨! 제발 다른데서 노세요! 이 방송은 랩배틀을 보려고 보는건데 정말 짜증나요! 이런거.원래.안 올리는데.이게.뭡니까?',
 '유쾌하고도 마냥 이쁜영화... 특히 여배우는 스타일 작렬ㅋ',
 '아 너무너무 재밋습니다~~~베스트키드',
 '나루토 극장판은 다 재미있다 !',
 '애무할때 진짜 남자꺼 쥐고 쪽쪽 거리네..시집 우째 갈라고 그러냐~',
 '별점주기도 아까운1人',
 '뻔한 주제를 식상한 방식으로 불필요한 선정성을 동원해 만든 상업영화임. 소설 고래를 쓴 천명관 소설가의 시나리오라니 믿기 힘듦. 척하는 이런 영화보다 차라리 헐리웃 블록버스터가 오히려 더 순수함.',
 '시대고발적인 영화치곤 너무 뛰어난 킬러 드라마',
 '대한민국 현실의 일부분을 보여준 좋은 영화',
 '기대에못미치는두스타의부진한연기력.억지스러운스토리전개',
 '너무 재미나게 봐서 비디오까지 샀어!!',
 '평이',
 '화려하고 멋있기만 해선 곤란한 비행.',
 '마지막을 생각해보게 되는 그런 영화인것같다ㅠㅠ',
 '어느 천재 청년의 따사로운 자아성장기...뭉클~',
 '흑백 아닌 컬러였다면 1점짜리 영화',
 '정말 재미없다.. 시간이 아깝다. 난 이런 개그는 별로고,이런 스토리 형식도 별로다. 어디가 웃기다는건지 모르겠다. 2번을 봐도 안웃겨. 그리고 영화 평가는 주관적일수밖에 없는데,재미없다하면 고등학교도 안나온거냐? 생각을하고 평점을달아라',
 '이영화는 지루한부분이 없습니다.',
 '배우들이 아깝다. 정크무비.',
 '캘리포니아는 미.맥 전쟁 이후에 미국에게 뺏긴땅.미국 제국주의로 조로를 망치다.',
 '재밌어요 ㅋㅋ 정다혜씨도 오랜만에 보고 ㅎㅎ 찌질함의 극을 보여주

In [51]:
y

tensor([1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
        1, 0, 1, 0, 0, 0, 1, 1])

In [52]:
encoded_x = tokenizer.batch_encode_plus(x, padding=True, return_tensors='pt')

In [53]:
encoded_x.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [54]:
encoded_x['input_ids']

tensor([[   101,   9685,  77884,  ...,      0,      0,      0],
        [   101,   9355, 118912,  ...,      0,      0,      0],
        [   101,   9576,  12508,  ...,      0,      0,      0],
        ...,
        [   101,   9477,  26444,  ...,      0,      0,      0],
        [   101,   9638,  19105,  ...,      0,      0,      0],
        [   101,   8932,  14423,  ...,      0,      0,      0]])

In [55]:
encoded_x['token_type_ids']

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [56]:
# '[CLS] 안녕 , 나는 고정호야 . [SEP] 나는 지금 공부를 하고 있어. [SEP]'
# token type 0: 1번째 문장
# token type 1: 2번째 문장 

In [57]:
encoded_x['attention_mask']

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [58]:
def train_epoch(model, dataloader, tokenizer, optimizer):
    model.train()
    train_loss = 0
    for i, (x,y) in enumerate(dataloader):
        # =========== TODO ============
        # 위에서 배운 내용을 바탕으로 빈칸을 채워보세요!
        # =============================
        x = tokenizer.batch_encode_plus(x, padding=True, return_tensors='pt')['input_ids'].to(DEVICE)
        y = y.to(DEVICE)
        loss = model(x, labels=y)['loss'] # [0]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if i % 50 == 0:
            print('Iter [{}/{}] Loss {:.6f}'.format(i+1, len(dataloader), train_loss / (i+1)))
    
    return train_loss / len(dataloader)

def test_epoch(model, dataloader, tokenizer):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
      for x,y in dataloader:
          # =========== TODO ============
          # 위에서 배운 내용을 바탕으로 빈칸을 채워보세요!
          # =============================
          x = tokenizer.batch_encode_plus(x, padding=True, return_tensors='pt')['input_ids'].to(DEVICE)
          out = model(x)['logits'] # [0] 
          pred = out.argmax(-1)
          preds.append(pred.cpu())
          labels.append(y)
    preds = torch.cat(preds)
    labels = torch.cat(labels)
    acc = (preds == labels).float().mean()
    print('ACC : {:.3f}'.format(acc))
    return preds, labels

def predict(model, tokenizer, sentence):
    model.eval()
    x = tokenizer.encode(sentence, return_tensors='pt').to(DEVICE)
    out = model(x)['logits']
    pred = out.argmax(-1)
    return pred.cpu()

In [59]:

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [60]:
EPOCHS=1

for i in range(EPOCHS):
    train_epoch(model, train_loader, tokenizer, optimizer)
    test_epoch(model, test_loader, tokenizer)

Iter [1/4999] Loss 0.713383
Iter [51/4999] Loss 0.690541
Iter [101/4999] Loss 0.677558
Iter [151/4999] Loss 0.645707
Iter [201/4999] Loss 0.635939
Iter [251/4999] Loss 0.610548
Iter [301/4999] Loss 0.592908
Iter [351/4999] Loss 0.579597
Iter [401/4999] Loss 0.563771
Iter [451/4999] Loss 0.551112
Iter [501/4999] Loss 0.540824
Iter [551/4999] Loss 0.529968
Iter [601/4999] Loss 0.522889
Iter [651/4999] Loss 0.516902
Iter [701/4999] Loss 0.511014
Iter [751/4999] Loss 0.503949
Iter [801/4999] Loss 0.498195
Iter [851/4999] Loss 0.492587
Iter [901/4999] Loss 0.488327
Iter [951/4999] Loss 0.484876
Iter [1001/4999] Loss 0.480648
Iter [1051/4999] Loss 0.476998
Iter [1101/4999] Loss 0.472264
Iter [1151/4999] Loss 0.468951
Iter [1201/4999] Loss 0.464703
Iter [1251/4999] Loss 0.461320
Iter [1301/4999] Loss 0.459058
Iter [1351/4999] Loss 0.455838
Iter [1401/4999] Loss 0.453079
Iter [1451/4999] Loss 0.450614
Iter [1501/4999] Loss 0.447829
Iter [1551/4999] Loss 0.445691
Iter [1601/4999] Loss 0.443646


In [61]:
predict(model, tokenizer, '이 영화는 최고야')

tensor([1])

In [62]:
predict(model, tokenizer, '이 영화는 별로야')

tensor([0])

In [63]:
test_epoch(model, test_loader, tokenizer)

ACC : 0.877


(tensor([0, 1, 1,  ..., 0, 1, 0]), tensor([1, 1, 1,  ..., 0, 0, 0]))